In [15]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

In [23]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [24]:
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown



def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:
# Used to securely store your API key
from google.colab import userdata

In [26]:
# Or use `os.getenv('HUGGINGFACEHUB_API_TOKEN')` to fetch an environment variable.
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = userdata.get("HF_TOKEN")
os.environ["HF_TOKEN"] = "HF_TOKEN"

Import Document

In [29]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/content/healthyheart.pdf")
doc = loader.load()

In [30]:
doc

[]

text splitting and chunking


In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(doc)

In [22]:
len(chunks)

0

In [11]:
chunks[0]

IndexError: list index out of range

In [ ]:
chunks[1]

Document(metadata={'source': '/content/sample_data/Data/healthyheart.pdf', 'page': 1}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute\nNIH Publication No. 06-5269\nDecember 2005')

In [ ]:
chunks[2]

Document(metadata={'source': '/content/sample_data/Data/healthyheart.pdf', 'page': 2}, page_content='U.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute\nWritten by: Marian Sandmaier')

In [ ]:
chunks[3]

Document(metadata={'source': '/content/sample_data/Data/healthyheart.pdf', 'page': 3}, page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3\nWhat Is Heart Disease?. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3')

In [ ]:
chunks[4]

Document(metadata={'source': '/content/sample_data/Data/healthyheart.pdf', 'page': 3}, page_content='Who Is at Risk? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nHow Risk Works . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4')

Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

<ipython-input-16-11fdde04d023>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Vector store - FAISS or ChromaDB

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore

In [ ]:
query = "who is at risk of heart disease"
search = vectorstore.similarity_search(query)

In [ ]:
to_markdown(search[0].page_content)

> The first step toward heart health is becoming aware of your 
> own personal risk for heart disease. Some risks, such as smoking
> cigarettes or being overweight, are obvious: All of us know whether
> we smoke or whether we need to lose a few pounds. But other risk

Retriever


In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [ ]:
retriever.get_relevant_documents(query)

<ipython-input-23-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 9, 'source': '/content/sample_data/Data/healthyheart.pdf'}, page_content='The first step toward heart health is becoming aware of your \nown personal risk for heart disease. Some risks, such as smoking\ncigarettes or being overweight, are obvious: All of us know whether\nwe smoke or whether we need to lose a few pounds. But other risk'),
 Document(metadata={'page': 9, 'source': '/content/sample_data/Data/healthyheart.pdf'}, page_content='The first step toward heart health is becoming aware of your \nown personal risk for heart disease. Some risks, such as smoking\ncigarettes or being overweight, are obvious: All of us know whether\nwe smoke or whether we need to lose a few pounds. But other risk'),
 Document(metadata={'page': 8, 'source': '/content/sample_data/Data/healthyheart.pdf'}, page_content='be changed. If your father or brother had a heart attack before age\n55, or if your mother or sister had one before age 65, you are more\nlikely to get heart dise

LLM - Opensource

In [13]:
#connect to google drive
from google.colab import drive

In [14]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
llm = LlamaCpp(
    model_path= "/content/drive/MyDrive/BioMistral-7B-SLERP/BioMistral-7B-SLERP.Q4_K_S.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral-7B-SLERP/BioMistral-7B-SLERP.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

RAG Chain

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = rag_chain.invoke("what disease affect the heart?")

llama_perf_context_print:        load time =   28962.75 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    58 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    80 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   85894.02 ms /   138 tokens


In [ ]:
to_markdown(response)

>  There are many diseases that can affect the heart. Some of the most common ones include coronary artery disease, heart failure, arrhythmias, and cardiomyopathy. It's important to note that there are many other conditions that can also affect the heart, so it's always a good idea to speak with a healthcare professional if you have any concerns about your heart health.

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ",result)

Input Prompt: how to take care of our heart?


Llama.generate: 41 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =   28962.75 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    20 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    53 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   43358.42 ms /    73 tokens


Answer:   There are several ways to take care of your heart, including eating a healthy diet, exercising regularly, maintaining a healthy weight, not smoking, and managing stress. It's also important to get regular check-ups with your doctor to monitor your heart health.
Input Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
